In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.metrics import BinaryAccuracy
from densenetmodel import create_model
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import cv2
import matplotlib.pyplot as plt
import random

  Using cached matplotlib-3.8.3-cp39-cp39-win_amd64.whl.metadata (5.9 kB)
  Using cached contourpy-1.2.0-cp39-cp39-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.49.0-cp39-cp39-win_amd64.whl.metadata (162 kB)
  Using cached kiwisolver-1.4.5-cp39-cp39-win_amd64.whl.metadata (6.5 kB)
  Using cached pillow-10.2.0-cp39-cp39-win_amd64.whl.metadata (9.9 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached importlib_resources-6.1.2-py3-none-any.whl.metadata (3.9 kB)
Using cached matplotlib-3.8.3-cp39-cp39-win_amd64.whl (7.6 MB)
Using cached contourpy-1.2.0-cp39-cp39-win_amd64.whl (181 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.49.0-cp39-cp39-win_amd64.whl (2.2 MB)
Using cached importlib_resources-6.1.2-py3-none-any.whl (34 kB)
Using cached kiwisolver-1.4.5-cp39-cp39-win_amd64.whl (56 kB)
Using cached pillow-10.2.0-cp39-cp39-win_amd64.whl (2.6 MB)
U

In [5]:
def plot_evaluation_metrics(history):
    # Plot accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

def plot_interpretability(images, predictions, class_names):
    # Select 5 random images
    random_indices = random.sample(range(len(images)), 5)
    selected_images = images[random_indices]
    selected_predictions = predictions[random_indices]

    # Plot the images and their predicted classes
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    for i, (image, prediction) in enumerate(zip(selected_images, selected_predictions)):
        axes[i].imshow(image)
        axes[i].axis('off')
        axes[i].set_title(f'Predicted Class: {class_names[prediction]}')
    plt.show()

In [13]:
def load_images(image_paths, image_size=(280, 280)):
    """
    This function loads grayscale images from disk based on their file paths and resizes them to a consistent size.
    """
    images = []

    for img_file in image_paths:
        # create the full input path and read the file
        image_path = os.path.join('Dataset/images/', img_file)
        
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # duplicate the grayscale image across three channels
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        images.append(image)

    return np.array(images)


def test_model(x_train, y_train, x_test, y_test):
    # dimensions of our images.
    img_width, img_height = 280, 280

    # specify the number of classes
    num_classes = 2

    # create the base pre-trained model
    # create the base pre-trained model
    model = create_model((img_width, img_height, 3), num_classes)


    # define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

    # fit the model
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping, learning_rate_reduction])

    # save weights to file
    model.save_weights('model_weights.h5')

    # predict the output 
    predictions = model.predict(x_test)

    # get the class with highest probability for each sample
    y_pred = np.argmax(predictions, axis=1)

    return y_pred

from keras.utils import to_categorical

In [14]:
def main():
    # load data
    x_train_paths = np.load('Dataset/X_train.npy', allow_pickle=True)
    y_train = np.load('Dataset/y_train.npy', allow_pickle=True)
    x_test_paths = np.load('Dataset/X_test.npy', allow_pickle=True)
    y_test = np.load('Dataset/y_test.npy', allow_pickle=True)

    x_train = load_images(x_train_paths)
    x_test = load_images(x_test_paths)

    # test model
    y_pred = test_model(x_train, y_train, x_test, y_test)

    # calculate accuracy
    accuracy = BinaryAccuracy()
    accuracy.update_state(y_test, y_pred)
    print('Test accuracy:', accuracy.result().numpy())

    class_names = [0, 1]  # Replace with your class names
    plot_interpretability(x_test, y_pred, class_names)

In [15]:
main()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
|